In [32]:
import re
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.svm import NuSVC
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_absolute_error

In [10]:
raw_data = pd.read_excel("Data_Pre-processing/urlset.xlsx")
raw_data.head()

,domain,ranking,mld_res,mld.ps_res,card_rem,ratio_Rrem,ratio_Arem,jaccard_RR,jaccard_RA,jaccard_AR,jaccard_AA,jaccard_ARrd,jaccard_ARrem,label,Unnamed: 14
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,10000000,1,0,18.0,107.611111,107.277778,0.0,0.0,0.0,0.0,0.8,0.795729,1,NaN
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,10000000,0,0,11.0,150.636364,152.272727,0.0,0.0,0.0,0.0,0,0.768577,1,NaN
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,10000000,0,0,14.0,73.500000,72.642857,0.0,0.0,0.0,0.0,0,0.726582,1,NaN
3,mail.printakid.com/www.online.americanexpress....,10000000,0,0,6.0,562.000000,590.666667,0.0,0.0,0.0,0.0,0,0.85964,1,NaN
4,thewhiskeydregs.com/wp-content/themes/widescre...,10000000,0,0,8.0,29.000000,24.125000,0.0,0.0,0.0,0.0,0,0.748971,1,NaN


In [3]:
print(len(raw_data.index))

11430


In [20]:
drop_columns = ['ranking','mld_res','mld.ps_res','card_rem','ratio_Rrem','ratio_Arem','jaccard_RR','jaccard_RA','jaccard_AA','jaccard_AR','jaccard_ARrd','jaccard_ARrem','Unnamed: 14']
raw_data = raw_data.drop(drop_columns, axis=1)
raw_data.head()

KeyError: "['ranking' 'mld_res' 'mld.ps_res' 'card_rem' 'ratio_Rrem' 'ratio_Arem'\n 'jaccard_RR' 'jaccard_RA' 'jaccard_AA' 'jaccard_ARrem' 'Unnamed: 14'] not found in axis"

In [23]:
raw_data = raw_data.drop(['jaccard_AR','jaccard_ARrd'], axis=1)

In [24]:
print(raw_data.columns)

Index(['domain', 'label'], dtype='object')


# Clean Data

In [25]:
clean_data = raw_data.dropna()
print(len(clean_data.index))

95915


In [31]:
clean_data["url_length"] = clean_data['domain'].str.len()
clean_data.head()

F:\applications\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,domain,label,url_length
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,1,225
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,1,81
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,1,177
3,mail.printakid.com/www.online.americanexpress....,1,60
4,thewhiskeydregs.com/wp-content/themes/widescre...,1,116


In [51]:
clean_data["num_per"] = clean_data['domain'].str.split('.').count()
#clean_data["num_per"] = clean_data["num_per"].count()
#clean_data.head()
clean_data.dtypes
clean_data.head()

F:\applications\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,domain,label,url_length,num_per
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,1,225,95915
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,1,81,95915
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,1,177,95915
3,mail.printakid.com/www.online.americanexpress....,1,60,95915
4,thewhiskeydregs.com/wp-content/themes/widescre...,1,116,95915


In [8]:
y = clean_data.status
feature_columns = ["length_url","https_token","nb_qm"]
X = clean_data[feature_columns]
train_X, val_X, train_y, val_y = train_test_split(X,y, random_state=1)

In [9]:
model = SVC(kernel = 'rbf',gamma = 'scale', shrinking = False)
model.fit(train_X, train_y)

SVC(shrinking=False)

In [10]:
val_predictions = model.predict(val_X)
print(val_predictions)
print(val_y.head())

['legitimate' 'phishing' 'phishing' ... 'legitimate' 'legitimate'
 'legitimate']
11091    legitimate
9789       phishing
384        phishing
10033    legitimate
3260       phishing
Name: status, dtype: object


In [11]:
def get_mae(X, y, model):
    train_X, val_X, train_y, val_y = train_test_split(X,y, random_state=1)
    print("Training Model...")
    model.fit(train_X, train_y)
    print("Making Predictions...")
    val_predictions = model.predict(val_X)
    print("Calculating MAE...")
    val_mae = mean_absolute_error(val_predictions, val_y)
    print(val_mae)

In [15]:
model.score(X,y)

0.6013998250218723

I think the MAE in this context represents the percentage chance the model will be wrong, as the target is a boolean. Let's try reducing the amount of features, and see how this affects the MAE

In [12]:
feature_columns = ["qty_dot_url","length_url",
                   "domain_length","qty_dot_directory","directory_length","file_length","qty_params","url_shortened"]
X = clean_data[feature_columns]
train_X, val_X, train_y, val_y = train_test_split(X,y, random_state=1)
model.fit(train_X, train_y)
val_predictions = model.predict(val_X)
val_mae = mean_absolute_error(val_predictions, val_y)
print(val_mae)

0.2009957713818033


Hmm, it seemed to have no effect? Lets try calculating the MAE when all columns are used as feature columns? I also created a function to calculate the MAE.

In [14]:
feature_for_columns = []
for column in clean_data.columns:
    feature_for_columns.append(column)
X=clean_data[feature_for_columns]
model = NuSVC(kernel = 'linear',gamma = 'scale', shrinking = False)
#get_mae(X,y,model)

In [15]:
feature_columns = ["qty_dot_url","qty_hyphen_url","qty_questionmark_url","qty_and_url","length_url",
                   "domain_length","qty_dot_directory","directory_length","file_length","qty_params","url_shortened"]
for feature in feature_columns:
    X=clean_data[feature_columns]
    model = NuSVC(kernel = 'linear',gamma = 'scale', shrinking = False)
    get_mae(X,y,model)
    feature_columns.remove(feature)
    print("Feature Removed: " + feature)

Training Model...
Making Predictions...
Calculating MAE...
0.2009957713818033
Feature Removed: qty_dot_url
Training Model...


KeyboardInterrupt: 

In [ ]:
feature_columns = ["length_url",
                   "directory_length","qty_params"]
for feature in feature_columns:
    X=clean_data[feature_columns]
    model = NuSVC(kernel = 'linear',gamma = 'scale', shrinking = False)
    get_mae(X,y,model)
    feature_columns.remove(feature)
    print("Feature Removed: " + feature)

In [ ]:
#pickle.dump(model, open("url_model.pkl","wb"))

In [ ]:
feature_columns = ["qty_dot_url"]
feature_columns_long = ["qty_hyphen_url","qty_questionmark_url","qty_and_url","length_url",
                   "domain_length","qty_dot_directory","directory_length","file_length","qty_params","url_shortened"]
for feature in feature_columns_long:
    X=clean_data[feature_columns]
    model = NuSVC(kernel = 'linear',gamma = 'scale', shrinking = False)
    get_mae(X,y,model)
    feature_columns.append(feature_columns_long[0])
    feature_columns_long.pop(0)
    print("Feature Added: " + feature)

In [ ]:
feature_columns = ["length_url","qty_params"]

These 3 features are all that is needed to train a model with a MAE of under 20% !
These are the features I'll be focusing on.

In [ ]:
new_df_columns = ["length_url",
                   "directory_length","qty_params", "phishing"]
new_df = raw_data[new_df_columns]
new_df.head(50)

In [ ]:
feature_columns = ["length_url","qty_params"]
X=clean_data[feature_columns]
model = NuSVC(kernel = 'linear',gamma = 'scale', shrinking = False)
get_mae(X,y,model)

In [20]:
feature_columns = ["length_url","qty_params"]
X=clean_data[feature_columns]
model = NuSVC(kernel = 'linear',gamma = 'scale', shrinking = False)
#get_mae(X,y,model)
model.fit(X,y)
model.score(X,y)

0.7918322107596556

Its even more accurate if we remove the length of the directory feature.
So the web app needs to:
- Parse the url
- Identify the URL length
- Identify the quantity of parameters
- Plug those features into the model, hosted on a Flask app

In [16]:
feature_columns = ["length_url","qty_params","qty_comma_url"]
X = clean_data[feature_columns]
train_X, val_X, train_y, val_y = train_test_split(X,y, random_state=1)

KeyError: "['qty_params', 'qty_comma_url'] not in index"

In [19]:
#pipe = make_pipeline(StandardScaler(), SVC(kernel = 'rbf',gamma = 'scale', shrinking = True, cache_size=1000,class_weight='balanced'))
model = SVC(kernel = 'rbf',gamma = 'auto', shrinking = True, cache_size=1000,class_weight='balanced')
model.fit(train_X,train_y)
#pipe.score(X,y)
pickle.dump(model, open("url_model2.pkl","wb"))

In [41]:
clf = GradientBoostingClassifier(n_estimators=112, learning_rate=1.0,
...     max_depth=1, random_state=0).fit(X, y)
clf.score(X,y)

0.8074857191576434

In [28]:
feature_columns = ["length_url","qty_params"]
X=clean_data[feature_columns]
model = LinearSVC()
model.fit(X,y)
model.score(X,y)

F:\applications\Anaconda\lib\site-packages\sklearn\svm\_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


0.7509591610537982